In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
cans = pd.read_csv('PittsburghSmartTrash.csv')
neighborhood = pd.read_csv('neighborhoods.csv')

FileNotFoundError: [Errno 2] File PittsburghSmartTrash.csv does not exist: 'PittsburghSmartTrash.csv'

In [8]:
fire_incident = pd.read_csv('pittsburgh_fire_incidents.csv')
neighborhood_area = pd.read_csv('neighborhoods.csv')

In [6]:
fire_incident['neighborhood'].value_counts()

Carrick                      255
Central Business District    225
Squirrel Hill South          225
South Side Flats             219
Hazelwood                    213
                            ... 
St. Clair                     11
East Carnegie                 10
Mt. Oliver                    10
Regent Square                  7
Mount Oliver Borough           6
Name: neighborhood, Length: 91, dtype: int64

In [9]:
sortedarea = neighborhood_area.sort_values(by = "hood")
sortedareadf = sortedarea[['hood','acres']]
newsortedareadf = sortedareadf.reset_index(drop=True)
newsortedareadf

,hood,acres
0,Allegheny Center,134.703
1,Allegheny West,90.841
2,Allentown,189.608
3,Arlington,302.320
4,Arlington Heights,84.592
...,...,...
85,Upper Lawrenceville,260.349
86,West End,128.679
87,West Oakland,138.372
88,Westwood,445.009


In [10]:
new_df = fire_incident.sort_values(["neighborhood"], ascending=True)
fire_incident['neighborhood'].value_counts()

newdf =fire_incident['neighborhood'].value_counts().rename_axis('neighborhood').reset_index(name='frequency')
sorteddf = newdf.sort_values(["neighborhood"], ascending=True)
sortednew = sorteddf.reset_index(drop=True)
mergesorteddata = sortednew.merge(newsortedareadf, left_on = 'neighborhood', right_on = 'hood', suffixes=('frequency', 'acres'))
mergesorteddata.sort_values(['frequency'], ascending=True)
mergesorteddata['ratio'] = mergesorteddata['acres'] / mergesorteddata['frequency']
ratio = mergesorteddata.sort_values(["ratio"], ascending=True)
ratio

,neighborhood,frequency,hood,acres,ratio
60,Perry North,100,Perry North,0.000,0.000000
46,Lincoln-Lemington-Belmar,172,Lincoln-Lemington-Belmar,180.105,1.047122
49,Marshall-Shadeland,119,Marshall-Shadeland,148.209,1.245454
43,Knoxville,146,Knoxville,193.982,1.328644
25,East Allegheny,111,East Allegheny,151.705,1.366712
...,...,...,...,...,...
81,Swisshelm Park,15,Swisshelm Park,308.575,20.571667
54,New Homestead,20,New Homestead,510.728,25.536400
31,Fairywood,23,Fairywood,622.746,27.075913
26,East Carnegie,10,East Carnegie,277.895,27.789500
